In [166]:
from IPython.display import HTML
HTML('<iframe src="http://pydata.org/nyc2014/schedule/" width=100% height=350></iframe>')

In [167]:
from bs4 import BeautifulSoup
import urllib2

# past events
# TODO: fetch this data from past events page
conferences = ['nyc2014', 'berlin2014', 'sv2014', 'ldn2014', 'nyc2013']

abstract_url = "http://pydata.org/%s/abstracts/"

conf_data = {}

# Collecting data about abstracts
for conference in conferences:
    print "loading data for %s conference" % conference
    raw = urllib2.urlopen(abstract_url % conference).read()
    soup = BeautifulSoup(raw)
    abstracts = [abstract.get_text().strip() for abstract in soup.find_all(class_="accordion-inner")]
    titles = [title.get_text().strip() for title in soup.find_all(class_="accordion-toggle")]
#     speakers = [speaker.get_text().strip() for speaker in soup.select(".accordion-heading h5 a")]
    
    conf_data[conference] = {}
    conf_data[conference]['abstracts'] = abstracts
    conf_data[conference]['titles'] = titles
#     conf_data[conference]['speakers'] = speakers

loading data for nyc2014 conference
loading data for berlin2014 conference
loading data for sv2014 conference
loading data for ldn2014 conference
loading data for nyc2013 conference


In [168]:
conf_data['nyc2014']['titles'][:20]

[u'(Easy), High Performance Text Processing with Rosetta',
 u'A Machine Learning Pipeline with Scikit-Learn',
 u'Advanced IPython Notebook Widgets',
 u'Advanced scikit-learn',
 u'Analyzing Satellite Images With Python Scientific Stack',
 u'Beautiful Interactive Visualizations in the Browser with Bokeh',
 u'Biological Data Science',
 u'Blaze Foundations: Part 1',
 u'Data Community/Meetup Organizers',
 u"Data Science: It's Easy as Py\u01c3",
 u'Data warehouse and conceptual modelling with Cubes 1.0',
 u'Data-driven conversations about biology',
 u'Decreasing Uncertainty with Weakly Informative Priors and Penalized Regression',
 u'Disco: Distributed Multi-Stage Data Pipelines',
 u'Driving Blaze in the Real World of Data Land Mines',
 u'Evaluating skills in educational and other settings: An overview',
 u'From DataFrame to Web Application in 10 minutes',
 u'Get To Know Your Data',
 u'Grids, Streets & Pipelines: Making a linguistic streetmap with scikit-learn',
 u'Healthcare Analytics']

In [169]:
from pandas import DataFrame

pydata = DataFrame()

for conf in conf_data:
    conf_dataframe = DataFrame.from_dict(conf_data[conf])
    conf_dataframe['conference'] = conf
    conf_dataframe['city'] = conf[:-4]
    conf_dataframe['year'] = int(conf[-4:])
    
    print DataFrame.head(conf_dataframe)
    
    pydata = pydata.append(conf_dataframe)
    

                                           abstracts  \
0                                                      
1  The The Greater Plains Collaborative (GPC) is ...   
2  To a lot of people, Facebook is a website for ...   
3  The ad targeting team at Yelp is tasked with p...   
4                                                      

                                              titles conference city  year  
0                                                        sv2014   sv  2014  
1  Using Python and Paver to Control a Large Medi...     sv2014   sv  2014  
2  A Full Stack Approach to Data Visualization: T...     sv2014   sv  2014  
3                               Ad Targeting at Yelp     sv2014   sv  2014  
4  Analyzing Satellite Images With Python Scienti...     sv2014   sv  2014  
                                           abstracts  \
0  The Python data ecosystem has grown beyond the...   
1  In this talk I will give an overview of Random...   
2  Clustering data is a fundament

In [170]:
print 'records in dataframe %i' % len(pydata)
pydata.groupby(['conference']).count(1).sort('year', ascending=False)

records in dataframe 233


,abstracts,titles,city,year
conference,,,,
berlin2014,50,50,50,50
nyc2014,50,50,50,50
sv2014,49,49,49,49
ldn2014,44,44,44,44
nyc2013,40,40,40,40


# Now we have all the data. Let's try to analyse it








In [171]:
import nltk
stop = nltk.corpus.stopwords.words('english')

text = {}
words = {}

In [172]:
stop_list = ["ll", "II", "ll", "http", "://", "e", "g", "2", "0"]

for conference in conf_data:
    raw = " ".join(conf_data[conference]['abstracts'])
    tokens = nltk.WordPunctTokenizer().tokenize(raw)
    text[conference] = nltk.Text(tokens)
    words[conference] = [w.lower() for w in text[conference] if w.lower() not in stop_list]
    words[conference] = [w for w in words[conference] if w not in stop]
    words[conference] = filter(lambda word: word not in u'%,-:()$\/;?.’–“”*\'[]', words[conference])
    words[conference] = [w for w in words[conference] if w not in ["ll", "II", "ll", "http", "://", "e", "g", "2", "0"]]

In [173]:
for conference in text:
    print conference
    print text[conference].collocations()
    print 
    

sv2014
http ://; nearest neighbor; machine learning; Reference Model;
neighbor algorithm; IPython Notebook; big data; open source; make
predictions; data analysis; Big Data; github repository; current
state; means clustering; visualization libraries; https ://; compiler
optimizations; accepting payments; block fraud; concise construction
None

ldn2014
http ://; machine learning; :// www; data processing; open source;
Matrix Factorisation; certain types; public clouds; rent ratios;
financial industry; PyData Boston; blocking technique; cloud
computing; exact solution; includes two; presentation focuses; drug
development; graphical plotting; quantum chemistry; wide range
None

berlin2014
http ://; machine learning; Big Data; Quantified Self; self tracking;
Semantic Web; Coming Soon; among others; open source; data analysis;
case study; Hadoop jobs; :// www; working knowledge; predictive model;
time permits; Add tranformations; Machine Learning; Operating System;
Pythonista interested
Non

In [174]:
numwords = {}
uniwords = {}

for conference in text:
    numwords[conference] = len(text[conference])
    uniwords[conference] = len(set(text[conference]))

print numwords
print uniwords

{'sv2014': 7462, 'ldn2014': 5859, 'nyc2013': 4328, 'berlin2014': 7633, 'nyc2014': 6293}
{'sv2014': 1906, 'ldn2014': 1738, 'nyc2013': 1337, 'berlin2014': 1877, 'nyc2014': 1824}


# Bigrams

In [176]:
from nltk.collocations import *

bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()


for conference in words:
    print "Bigrams " + str(conference)
    finder = BigramCollocationFinder.from_words(words[conference])
    scored = finder.score_ngrams(bigram_measures.raw_freq)
    print pd.DataFrame(scored[:15])
    print "\n\n"

Bigrams sv2014
                        0         1
0             (big, data)  0.002805
1     (machine, learning)  0.002550
2          (open, source)  0.002295
3         (scikit, learn)  0.002295
4     (ipython, notebook)  0.002040
5     (high, performance)  0.001785
6        (data, analysis)  0.001530
7         (data, science)  0.001530
8           (high, level)  0.001530
9            (k, nearest)  0.001530
10    (nearest, neighbor)  0.001530
11  (neighbor, algorithm)  0.001530
12           (real, time)  0.001530
13     (reference, model)  0.001530
14         (using, bokeh)  0.001530



Bigrams ldn2014
                           0         1
0         (data, processing)  0.001969
1        (machine, learning)  0.001969
2            (data, science)  0.001312
3                (data, set)  0.001312
4             (open, source)  0.001312
5                (python, r)  0.001312
6             (python, used)  0.001312
7            (scikit, learn)  0.001312
8              (use, python)  0.001312


In [177]:
finder = BigramCollocationFinder.from_words(words)
scored = finder.score_ngrams(bigram_measures.raw_freq)

In [178]:
scored

[(('berlin2014', 'nyc2013'), 0.2),
 (('ldn2014', 'berlin2014'), 0.2),
 (('nyc2013', 'nyc2014'), 0.2),
 (('sv2014', 'ldn2014'), 0.2)]